In [3]:
import numpy as np
import itertools
import time
import math
from functools import reduce # python3 compatibility
from operator import mul
import json
import copy
import networkx as nx
import matplotlib.pyplot  as plt
from random import sample

In [4]:
# Function will generate iterator of value with 3 conditions
# 1, value item in values
# 2, sum of array <= max sum
# 3, sum of array >= min sum
def gen_iter_with_min_max_sum_cond(values, len_new_arr, min_sum, max_sum):
    new_arr = []
    min_val = min(values)
    max_val = max(values)
    while len(new_arr) < len_new_arr:
        rand_val = np.random.choice(values)
        remain_min_sum = min_sum - rand_val
        remain_max_sum = max_sum - rand_val
        remain_locationNum_new_arr = len_new_arr - len(new_arr) - 1

        if remain_locationNum_new_arr * max_val >= remain_min_sum and remain_locationNum_new_arr * min_val <= remain_max_sum:
            new_arr.append(rand_val)
            min_sum = remain_min_sum
            max_sum = remain_max_sum
    return iter(new_arr)

In [5]:
def gen_tasks(task_num, max_capNum, capabilities): # task_num is the number of task, max_capNum is the maximum number of cap a task could require
    return [sorted(np.random.choice(capabilities,np.random.randint(3,max_capNum+1),replace=False)) for j in range(0, task_num)]

In [6]:
def gen_tasks_cap(tasks, task_capVal, capNum, min_sum_task_caps, max_sum_task_caps):
    tasks_cap = []
    for task in tasks:
        capNum_in_task = len(task)
        avail_caps_in_task = gen_iter_with_min_max_sum_cond(task_capVal, capNum_in_task, min_sum_task_caps, max_sum_task_caps)
        tasks_cap.append([next(avail_caps_in_task) if cap in task else 0 for cap in range(0, capNum)])
    return tasks_cap

In [7]:
def gen_constraints(agent_num, task_num, agent_managers_index): #power is the inforce you put in the probabilities
    # the maximum tasks an agent could work on depends on the number of tasks available (e.g, if |T| = 1/2|A|, the roughly each agent can work on two tasks)
    # calculate the max and min edges for agents
    a_taskInds = [[j for j in range(0, task_num)] for i in range(0,agent_num)]
    
    #asign agent manager
    for index, agent_manager_index in enumerate(agent_managers_index):
        a_taskInds[agent_manager_index] = [index]

    # get also the list of agents for each task   
    t_agents = [[i for i in range(0,agent_num) if j in a_taskInds[i]] for j in range(0,task_num)]
    
    return a_taskInds, t_agents

In [8]:
def gen_agents(a_taskInds, tasks, max_capNum, capabilities, agent_capVal, min_sum_agent_caps, max_sum_agent_caps): # m is the number of task, max_capNum is the maximum number of cap a task could require, max_capVal is the maximum capability value
    agent_num = len(a_taskInds)
    caps_lists = []
    contri_lists = []
    for i in range(0,agent_num):
        t_caps = [tasks[j] for j in a_taskInds[i]] # lists of caps that each task agent could perform
        caps_union = set(itertools.chain(*t_caps)) # union of caps of tasks that agent could perform
        a_cap_num = np.random.randint(min(3,max_capNum,len(caps_union)),min(len(caps_union),max_capNum)+1) # the num of caps the agent will have
        a_caps = set([np.random.choice(t_c) for t_c in t_caps]) # initial draw to guarantee the agent has some contribution to each of the task that he could do
        rest_choices = list(caps_union.difference(a_caps))
        if rest_choices != []:
            update_len = max(0, a_cap_num - len(a_taskInds[i]))
            a_caps.update(np.random.choice(rest_choices, min(len(rest_choices), update_len), replace=False))

        # Generate capabilities value in agent i
        caps_lists.append(sorted(list(a_caps)))
        capNum_in_agent = len(a_caps)
        avail_caps_in_agent = gen_iter_with_min_max_sum_cond(agent_capVal, capNum_in_agent, min_sum_agent_caps, max_sum_agent_caps)
        contri_lists.append([next(avail_caps_in_agent) if cap in a_caps else 0 for cap in range(0, len(capabilities))])
       
    return caps_lists, contri_lists

In [9]:
def cal_task_util(groups, agents, tasks, min_distance, capNum):
    task_num = len(tasks)
    agent_num = len(agents)
    result = 0
    for task_index in range(0, task_num):
        curr_min_distance_group = min_distance[task_index]
        curr_group = []
        curr_group.extend(groups[task_index])

        supplied_cap = [0 for i in range(0, capNum)]
        required_cap = tasks[task_index]

        commuincation_cost = 0

        for agent_index in range(0, agent_num):
            if agent_index in curr_group:
                commuincation_cost += curr_min_distance_group[agent_index]
                for cap_index in range(0, capNum):
                    supplied_cap[cap_index] += agents[agent_index][cap_index]
        
        is_task_complete = True

        sum_of_supplied_cap = 0
        sum_of_required_cap = sum(required_cap)

        for cap_index in range(0, capNum):
            if supplied_cap[cap_index] < required_cap[cap_index]:
                is_task_complete = False
                sum_of_supplied_cap += supplied_cap[cap_index]
            else:
                sum_of_supplied_cap += required_cap[cap_index]

        res_satis = sum_of_required_cap if is_task_complete else sum_of_supplied_cap/(math.exp(sum_of_supplied_cap/sum_of_required_cap))
        result += (res_satis - commuincation_cost)

    return result

In [10]:
def cal_contribute(agents, agent_index, tasks, task_index, groups, group_index, min_distance, capNum):
    groups_without_agent = []
    groups_without_agent.extend(groups)

    C_without_agent = cal_task_util(groups_without_agent, agents, tasks, min_distance, capNum)

    groups_with_agent = []
    groups_with_agent = copy.deepcopy(groups)

    for gr_index in range(0, len(groups)):
        if gr_index == group_index:
            groups_with_agent[gr_index].append(agent_index)
        elif agent_index in groups_with_agent[gr_index]:
            groups_with_agent[gr_index].remove(agent_index)
            
    C_with_agent = cal_task_util(groups_with_agent, agents, tasks, min_distance, capNum)

    return C_with_agent - C_without_agent

In [11]:
def print_groups(groups):
    print_groups = copy.deepcopy(groups)
    for group_index in range(0, len(print_groups)):
        print_groups[group_index] = sorted(print_groups[group_index])
        for item_index in range(0, len(groups[group_index])):
            print_groups[group_index][item_index] += 1
    return print_groups

In [12]:
def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

In [13]:
def append_record(record, filename, typ):
    with open(filename, 'a') as f:
        if typ != '':
            json.dump(record, f, default = typ)
        else:
            json.dump(record, f, default=convert)
        f.write('\n')
        f.close()

In [14]:
def greedy(agents, tasks, constraints, tasks_cap, min_distance, agent_managers_index, agent_contractors_index, alloc = []):
    a_taskInds = constraints[0]
    agent_num = len(agents)
    task_num = len(tasks)
    cap_num = len(tasks[0])

    alloc = [task_num for i in range(0, agent_num)] if alloc == [] else alloc
    groups = [[] for i in range(0, task_num+1)]
    move_vals = [[0 for task_index in range(0, task_num+1)] for agent_index in range(0, agent_num)]
    
    # Assign each task with one agent manager
    for agent_manager_index in agent_managers_index:
        alloc[agent_manager_index] = a_taskInds[agent_manager_index][0]
        move_vals[agent_manager_index] = [-1000 for j in range(0, task_num + 1)]
    
    # Add agent into group
    for i in range(0, len(alloc)):
        groups[alloc[i]].append(i)

    for agent_index in agent_contractors_index:
        move_vals[agent_index] = [ 0 if task_index == alloc[agent_index] else (round(cal_contribute(agents, agent_index=agent_index, tasks=tasks_cap, task_index=task_index, groups=groups, group_index=task_index, min_distance=min_distance, capNum=cap_num) - cal_contribute(agents, agent_index=agent_index, tasks=tasks_cap, task_index=alloc[agent_index], groups=groups, group_index=alloc[agent_index], min_distance=min_distance, capNum=cap_num), 2)) for task_index in range(0, task_num + 1) ]    
    
    # Init communication val
    iteration = 0
    max_moveIndexs = [np.argmax([move_vals[i][j] for j in a_taskInds[i]]+[0]) for i in range(0,agent_num)]
    max_moveVals = [move_vals[i][a_taskInds[i][max_moveIndexs[i]]] if max_moveIndexs[i] < len(a_taskInds[i]) 
                    else move_vals[i][task_num]
                                 for i in range(0,agent_num)]

    while (True):
        iteration += 1
        feasible_choices = [i for i in agent_contractors_index if max_moveVals[i]>0]
        if feasible_choices ==[]:
            break
        else:
            a_index = np.argmax(max_moveVals)
            t_index  = a_taskInds[a_index][max_moveIndexs[a_index]] if max_moveIndexs[a_index]< len(a_taskInds[a_index]) else task_num
        
        # perfom move
        old_t_index = alloc[a_index] 
        alloc[a_index] = t_index
        groups[old_t_index].remove(a_index)
        groups[t_index].append(a_index)
        
        for agent_index in agent_contractors_index:
            move_vals[agent_index] = [ 0 if task_index == alloc[agent_index] else (round(cal_contribute(agents, agent_index=agent_index, tasks=tasks_cap, task_index=task_index, groups=groups, group_index=task_index, min_distance=min_distance, capNum=cap_num) - cal_contribute(agents, agent_index=agent_index, tasks=tasks_cap, task_index=alloc[agent_index], groups=groups, group_index=alloc[agent_index], min_distance=min_distance, capNum=cap_num), 2)) for task_index in range(0, task_num + 1) ] 

        max_moveIndexs = [np.argmax([move_vals[i][j] for j in a_taskInds[i]]+[0]) for i in range(0,agent_num)]
        max_moveVals = [move_vals[i][a_taskInds[i][max_moveIndexs[i]]] if max_moveIndexs[i] < len(a_taskInds[i]) 
                        else move_vals[i][task_num]
                                    for i in range(0,agent_num)]

    return alloc, cal_task_util(groups, agents, tasks_cap, min_distance, cap_num), iteration

In [15]:
def random(agents, tasks, constraints, min_distance, cap_num, agent_managers_index):
    task_num = len(tasks)
    agent_num = len(agents)
    a_taskInds = constraints[0]
    alloc = [np.random.choice(a_taskInds[i]+[task_num]) for i in range(0,agent_num)]
    for agent_manager_index in agent_managers_index:
        alloc[agent_manager_index] = a_taskInds[agent_manager_index][0]
        
    groups = [[] for i in range(0, task_num+1)]
    # Add agent into group
    for i in range(0, len(alloc)):
        groups[alloc[i]].append(i)
    return alloc, cal_task_util(groups, agents,tasks, min_distance, cap_num)

In [21]:
def gen_random_graph(num_of_points, min_weight, max_weight, agent_managers_index, agent_contractors_index):
    graph_connected = False
    agent_manager_connections_num = 4
    agent_contractor_connections_num = 2
    while (True):
        G = nx.Graph()
        for agent_manager in agent_managers_index:
            targets = sample(range(0, num_of_points), agent_manager_connections_num)
            for target in targets:
                if nx.is_path(G, [agent_manager, target]) is False:
                    weight = np.random.randint(min_weight, max_weight + 1)
                    nx.add_path(G, [agent_manager, target], weight=weight)
        for agent_contractor in agent_contractors_index:
            targets = sample(range(0, num_of_points), agent_contractor_connections_num)
            for target in targets:
                if nx.is_path(G, [agent_contractor, target]) is False:
                    weight = np.random.randint(min_weight, max_weight + 1)
                    nx.add_path(G, [agent_contractor, target], weight=weight)
        graph_connected = nx.is_connected(G)
        if graph_connected is False:
            continue
        else:
            return G

In [18]:
#distance_agent has construc [min_to_agent_0, min_to_agent_1, ...., min_to_agent_n]
def get_communicate_cost_from_agent_manager(networkx_graph, agent_manager, agent_num):
    length, shortest_path = nx.single_source_dijkstra(networkx_graph, agent_manager, weight="weight")
    return [length[agent_index] for agent_index in range(0, agent_num)]

In [19]:
# Not use in big graphs
def draw_graph(G):
    pos = nx.spring_layout(G, seed=7)  # positions for all nodes - seed for reproducibility
    nx.draw_networkx_nodes(G, pos, node_size=700) # nodes
    nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")   # node labels

    edge = [(u, v) for (u, v, d) in G.edges(data=True)]   # edges
    nx.draw_networkx_edges(G, pos, edgelist=edge, width=6)

    edge_labels = nx.get_edge_attributes(G, "weight")   # edge weight labels
    nx.draw_networkx_edge_labels(G, pos, edge_labels)

    ax = plt.gca()
    ax.margins(0.08)
    plt.tight_layout()
    plt.show()

In [23]:
run_num = 10
min_t_num = 2
max_t_num = 11
step = 1
ex_identifier=0

#  conditions of Tasks generator
task_capVal = [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
min_sum_task_caps = 45
max_sum_task_caps = 50

# conditions of Agents generator
agent_capVal = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
min_sum_agent_caps = 20
max_sum_agent_caps = 30

#conditions of Graph generator
min_weight_edge = 2
max_weight_edge = 6

capNum = 5
a_min_edge = 1
t_max_edge = 5
capabilities = list(range(0,capNum))

for run in range(0, run_num):
    for task_num in range(min_t_num, max_t_num + 1, step):
        ex_identifier += 1

        agent_num = 3 * task_num
        agent_managers_index = sorted(sample(range(0, agent_num), task_num))
        agent_contractors_index = [agent_index for agent_index in range(0, agent_num) if agent_index not in agent_managers_index]
        result = {"ex_identifier":ex_identifier,"task_num":task_num,"agent_num":agent_num}
        print('ex_identifier',ex_identifier,'task_num:',task_num,'  agent_num:', agent_num)

        tasks = gen_tasks(task_num, capNum, capabilities)
        tasks_cap = gen_tasks_cap(tasks, task_capVal, capNum, min_sum_task_caps, max_sum_task_caps)
        constraints = gen_constraints(agent_num ,task_num, agent_managers_index)
        a_taskInds = constraints[0]
        agents_cap, agents = gen_agents(a_taskInds,tasks, capNum, capabilities, agent_capVal, min_sum_agent_caps, max_sum_agent_caps)

        # RANDOM GRAPH
        g = gen_random_graph(agent_num, min_weight_edge, max_weight_edge, agent_contractors_index, agent_managers_index)

        # DATA IN DOCUMENT
        # g = nx.Graph()
        # g.add_weighted_edges_from([(0, 1, 2), (0, 5, 2), (0,4,4), (5,4,6), (5,2,3),(4,2,2),(4,3,2),(4,1,3)], "weight")
        # draw_graph(g)

        min_distance = [get_communicate_cost_from_agent_manager(g, agent_manager_index, agent_num) for agent_manager_index in agent_managers_index]

        # Test with data in doc
        # agents = [[10,5,5], [10,5,15], [5,5,10], [10,5,10], [10,5,5],[10,5,5]]
        # tasks_cap = [[20,10,20],[15,15,15]]
        start= time.time()
        r = greedy(agents,tasks,constraints, tasks_cap, min_distance, agent_managers_index, agent_contractors_index)
        end=time.time()

        result['g'] = round(r[1], 2)
        result['g_iter'] = r[2]
        result['g_t'] = end - start

        print("eGreedy time:", result['g_t'],'result:',result['g'], 
              "iteration:",result['g_iter'])
        
        start= time.time()
        r = random(agents,tasks_cap,constraints, min_distance, capNum, agent_managers_index)
        end=time.time()

        alloc = r[0]
        result['rand'] = round(r[1], 2)
        result['rand_t'] = end - start
        print("rand time:", result['rand_t'],'result:',result['rand'])

        start= time.time()
        r = greedy(agents,tasks,constraints, tasks_cap, min_distance, agent_managers_index, agent_contractors_index, alloc)
        end=time.time()

        result['rand_g'] = round(r[1], 2)
        result['rand_g_iter'] = r[2]
        result['rand_g_t'] = end - start
        
        print("rand Greedy time:", result['rand_g_t'],'result:',result['rand_g'], 
             "iteration:",result['g_iter'])
        
        print()
        
        files = {'result_cap':[result,'']}
        for filename in list(files.keys()):
            append_record(files[filename][0],filename,typ = files[filename][1])

ex_identifier 1 task_num: 2   agent_num: 6
eGreedy time: 0.0010228157043457031 result: 59.3 iteration: 2
rand time: 0.0 result: 45.2
rand Greedy time: 0.0010831356048583984 result: 78 iteration: 2

ex_identifier 2 task_num: 3   agent_num: 9
eGreedy time: 0.0029740333557128906 result: 94.84 iteration: 3
rand time: 0.0 result: 54.23
rand Greedy time: 0.0019774436950683594 result: 91.82 iteration: 3

ex_identifier 3 task_num: 4   agent_num: 12
eGreedy time: 0.013810396194458008 result: 105.49 iteration: 5
rand time: 0.0 result: 20.9
rand Greedy time: 0.020914554595947266 result: 115.25 iteration: 5

ex_identifier 4 task_num: 5   agent_num: 15
eGreedy time: 0.030751705169677734 result: 184.62 iteration: 6
rand time: 0.0 result: 36.84
rand Greedy time: 0.054561614990234375 result: 169.62 iteration: 6

ex_identifier 5 task_num: 6   agent_num: 18
eGreedy time: 0.05155467987060547 result: 244.99 iteration: 6
rand time: 0.0 result: 118.19
rand Greedy time: 0.07670903205871582 result: 244 iterat

KeyboardInterrupt: 